# Detecção de Raças de Cachorros utilizando Redes Neurais

## Importações

### Importando biblioteca

In [22]:

from sys import platform
import os
import pandas as pd
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Importando Dataset

In [23]:
if platform == "linux" or platform == "linux2":
    folder_path = os.getcwd() + "/" + "low-resolution"
elif platform == "win32":
   folder_path = os.getcwd() + "\\" + "low-resolution"


list_images = []

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            
            file_path = os.path.join(subfolder_path, file)
            image_about = {}
            breed = subfolder.split("-")[-1]
            image_about["breed"] = breed
            image_about["filename"] = file
            image_about["path_file"] = file_path
            list_images.append(image_about)

df_images = pd.DataFrame.from_records(list_images)

### Pré-processamento

In [24]:
if platform == "linux" or platform == "linux2":
    os_path = "/" 
elif platform == "win32":
   os_path = "\\"

folder_path = os.getcwd() + os_path + "low-resolution"
slot_size = 20

#Retorna uma tupla com os path de cada imagem e o target randomizados
def load_path_of_dataset():
    arquivos = os.listdir(folder_path)
    subpastas = [subpasta for subpasta in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subpasta))]
    imagens_path = []
    labels = []
    for subpasta in subpastas:
        subpasta_path = os.path.join(folder_path, subpasta)
        imagens = os.listdir(subpasta_path)
        imagens_path.extend([subpasta + os_path + s for s in imagens])
        label = subpasta.split("-")[2]
        labels.extend([label for s in imagens])
    minha_lista = list(zip(imagens_path, labels))
    random.shuffle(minha_lista)
    return minha_lista

#função responsavel em realizar a divisão em slots
#recebe como parametro a tupla da função load_path_of_dataset, e o numero da interação para carregar o dataframe
#retorna uma tupla menor, e o numero máximo de interações posiveis até o fim do dataset
def slot_loading(interacao, imagens_amostradas, imagens_labels):
    if interacao <= 0:
        raise ValueError('O valor da interação, deve ser maior que zero ! ')
    elif interacao > (interacao * len(imagens_amostradas)):
        raise ValueError('O valor da interação, é muito grande !')
    
    start_index = (interacao - 1) * slot_size
    end_index = interacao * slot_size
    if end_index > len(imagens_amostradas):
        return imagens_amostradas[start_index:], imagens_labels[start_index:]

    return imagens_amostradas[start_index:end_index], imagens_labels[start_index:end_index], len(imagens_amostradas) // slot_size

#Recebe como parametro uma tupla
#retorna um dataset já pré-processado e formatados
def get_images(imagens_amostradas):
    dataset = pd.DataFrame(columns=['Image', 'Label'])
    img_size_height = 300
    img_size_width = 400
    for imagem_nome in imagens_amostradas:
        # extensao = os.path.splitext(imagem_nome)[1].lower()
        imagem_path = os.path.join(folder_path, imagem_nome[0])
        imagem = Image.open(imagem_path)
        imagem = imagem.resize((img_size_width, img_size_height))  # Redimensiona para um tamanho comum
        imagem_array = np.array(imagem) / 255.0  # Normaliza dividindo por 255
        dataset.loc[len(dataset)] = [imagem_array, imagem_nome[1]]

    return dataset;


#### Exemplo de utilização

In [25]:

# path_dataset = load_path_of_dataset()
# slot_path_dataset, interations = slot_loading(1,path_dataset)
# print("Interations: ", interations)
# dataset = get_images(slot_path_dataset);
# print(dataset)

## Separação do Conjunto de teste e treino

In [26]:
# Sua tupla
path_dataset = load_path_of_dataset()

# Separando o conjunto de teste e treino
X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla

# Separando 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Exibindo as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", len(X_treino), len(y_treino))
print("Dimensões do conjunto de teste:", len(X_teste), len(y_teste))

Dimensões do conjunto de treino: 49302 49302
Dimensões do conjunto de teste: 21130 21130


## Criação do modelo

In [32]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# Model Architecture
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133,activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        8256      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 64)       

2023-07-17 20:27:17.379700: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2023-07-17 20:27:17.391182: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2023-07-17 20:27:17.397419: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.


#### Treinamento em batches 

In [33]:

# X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
# y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla

# Treinar o modelo
x_new, y_new, i = slot_loading(1, X_treino, y_treino)
# model.fit(x_new, y_new)

for o in range(1, i):
    print( o," of " ,i)
    x_new, y_new = None, None
    x_new, y_new, i = slot_loading(o, X_treino, y_treino)
    x_new = get_images(x_new)
    print(x_new)
    print(y_new)
    print(i)
    model.fit(x_new , y_new, 
               shuffle = True,
               batch_size = 20,
               epochs = 25,
               verbose = 1)
# # Avaliar o modelo
# score = model.score(dados_teste, rotulos_teste)
# print("Acurácia do modelo:", score)




1  of  2465
['1936-n000005-Pomeranian/n105168.jpeg', '7449-n000128-teddy/n166300.jpeg', '5355-n000126-golden_retriever/n156700.jpeg', '207-n000036-Irish_terrier/n113750.jpg', '235-n000097-clumber/n128459.jpg', '209-n000049-kelpie/n116148.jpg', '2192-n000088-Samoyed/n125110.jpeg', '220-n000069-Mexican_hairless/n120393.jpg', '7449-n000128-teddy/n166842.jpeg', '257-n000108-Old_English_sheepdog/n133341.jpg', '1936-n000005-Pomeranian/n105294.jpg', '229-n000087-Kerry_blue_terrier/n126167.jpg', '234-n000091-Appenzeller/n127589.jpg', '2192-n000088-Samoyed/n124316.jpeg', '2925-n000114-toy_poodle/n140419.jpg', '7449-n000128-teddy/n162279.jpeg', '3083-n000117-Bichon_Frise/n144182.jpg', '5355-n000126-golden_retriever/n160829.jpeg', '235-n000097-clumber/n128481.jpg', '2594-n000109-Border_collie/n135033.jpeg']
['Pomeranian', 'teddy', 'golden_retriever', 'Irish_terrier', 'clumber', 'kelpie', 'Samoyed', 'Mexican_hairless', 'teddy', 'Old_English_sheepdog', 'Pomeranian', 'Kerry_blue_terrier', 'Appenzell

ValueError: in user code:

    File "/home/max/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/max/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/max/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/max/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/max/.local/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/max/.local/lib/python3.8/site-packages/keras/src/engine/input_spec.py", line 253, in assert_input_compatibility
        raise V